<a href="https://colab.research.google.com/github/Ponnraj0301/JUDICIO/blob/main/judicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1 — Install needed packages
!pip install -q flask flask-cors pyngrok nest_asyncio requests
!pip install -q flask flask-cors pyngrok nest_asyncio requests

In [ ]:
# Cell 2 — Imports and set API key (you will be prompted)
import os
from getpass import getpass
import json
import time
import uuid
from threading import Thread

# Prompt for your OpenAI API key (this won't display the key)
OPENAI_API_KEY = getpass("Paste your OpenAI API key (it will be hidden): ").strip()
if not OPENAI_API_KEY:
    raise SystemExit("No API key provided — stop and re-run the cell with a key.")

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# You can change this model if you have access to different models
MODEL = "gpt-4o-mini"   # change to "gpt-4o", "gpt-5", etc., if available
print("Model set to:", MODEL)


Paste your OpenAI API key (it will be hidden): ··········
Model set to: gpt-4o-mini


In [ ]:
# Cell 3 — local law data & helper
lawData = [

  {
    "act": "Code of Civil Procedure, 1908",
    "section": "2-43",
    "description": "General provisions, definitions, and powers of civil courts."
  },
  {
    "act": "Code of Civil Procedure, 1908",
    "section": "9",
    "description": "Courts' jurisdiction to try all civil suits unless specifically barred."
  },
  {
    "act": "Code of Civil Procedure, 1908",
    "section": "11",
    "description": "Prevents a party from filing a case that has already been decided (res judicata)."
  },
  {
    "act": "Code of Civil Procedure, 1908",
    "section": "80",
    "description": "Requires notice to be given before suing the government or public authority."
  },
  {
    "act": "Code of Civil Procedure, 1908",
    "section": "96-110",
    "description": "Appeals and revisions from lower courts."
  },
  {
    "act": "Code of Civil Procedure, 1908",
    "section": "151",
    "description": "Courts' inherent powers to prevent injustice."
  },
  {
    "act": "Code of Civil Procedure, 1908",
    "section": "Orders I-XXVI",
    "description": "Detailed procedural rules: pleadings, summons, trial, discovery, and execution."
  },
  {
    "act": "Indian Evidence Act, 1872",
    "section": "3-21",
    "description": "Defines relevant facts and what evidence is legally relevant in civil and criminal cases."
  },
  {
    "act": "Indian Evidence Act, 1872",
    "section": "24-30",
    "description": "Deals with confessions, both voluntary and involuntary."
  },
  {
    "act": "Indian Evidence Act, 1872",
    "section": "32-33",
    "description": "Dying declarations and statements by persons unable to testify."
  },
  {
    "act": "Indian Evidence Act, 1872",
    "section": "45-51",
    "description": "Expert evidence in technical or professional matters."
  },
  {
    "act": "Indian Evidence Act, 1872",
    "section": "65-73",
    "description": "Documentary evidence: How documents are proved in court."
  },
  {
    "act": "Indian Evidence Act, 1872",
    "section": "101-104",
    "description": "Burden of proof: Who must prove the facts."
  },
  {
    "act": "Indian Evidence Act, 1872",
    "section": "114-117",
    "description": "Presumptions of law: Legal assumptions in absence of evidence."
  },
  {
    "act": "Commercial Courts Act, 2015",
    "section": "2",
    "description": "Defines 'commercial dispute' and 'commercial court'."
  },
  {
    "act": "Commercial Courts Act, 2015",
    "section": "3",
    "description": "Establishes commercial courts and divisions."
  },
  {
    "act": "Commercial Courts Act, 2015",
    "section": "4",
    "description": "Jurisdiction of commercial courts."
  },
  {
    "act": "Commercial Courts Act, 2015",
    "section": "5",
    "description": "Minimum claim threshold (₹1 crore) for commercial courts."
  },
  {
    "act": "Commercial Courts Act, 2015",
    "section": "6",
    "description": "Transfer of cases to commercial courts if necessary."
  },
  {
    "act": "Commercial Courts Act, 2015",
    "section": "10",
    "description": "Expedited resolution of commercial disputes."
  },
  {
    "act": "Commercial Courts Act, 2015",
    "section": "11",
    "description": "Appeals and revisions of commercial court orders."
  },
  {
    "act": "Companies Act, 2013",
    "section": "2-6",
    "description": "Definitions, types of companies, share capital, and corporate structure."
  },
  {
    "act": "Companies Act, 2013",
    "section": "7",
    "description": "Process of company incorporation."
  },
  {
    "act": "Companies Act, 2013",
    "section": "248",
    "description": "Strike-off of inactive companies."
  },
  {
    "act": "Companies Act, 2013",
    "section": "271",
    "description": "Penalties for fraud or misconduct by company officers."
  },
  {
    "act": "Companies Act, 2013",
    "section": "92-137",
    "description": "Financial reporting, audits, and annual accounts."
  },
  {
    "act": "Insolvency and Bankruptcy Code, 2016",
    "section": "7",
    "description": "Initiation of corporate insolvency resolution by financial creditors."
  },
  {
    "act": "Insolvency and Bankruptcy Code, 2016",
    "section": "9",
    "description": "Initiation by operational creditors."
  },
  {
    "act": "Insolvency and Bankruptcy Code, 2016",
    "section": "12",
    "description": "Insolvency resolution must be completed within 330 days."
  },
  {
    "act": "Insolvency and Bankruptcy Code, 2016",
    "section": "29",
    "description": "Resolution process and submission of plans."
  },
  {
    "act": "Insolvency and Bankruptcy Code, 2016",
    "section": "52-60",
    "description": "Liquidation process if resolution fails."
  },
  {
    "act": "Arbitration and Conciliation Act, 1996",
    "section": "7",
    "description": "Recognition and validity of arbitration agreements."
  },
  {
    "act": "Arbitration and Conciliation Act, 1996",
    "section": "9-14",
    "description": "Interim measures by courts in arbitration."
  },
  {
    "act": "Arbitration and Conciliation Act, 1996",
    "section": "15-34",
    "description": "Arbitration proceedings, awards, and enforcement."
  },
  {
    "act": "Arbitration and Conciliation Act, 1996",
    "section": "35-37",
    "description": "Appointment and powers of arbitrators."
  },
  {
    "act": "Arbitration and Conciliation Act, 1996",
    "section": "42-45",
    "description": "Conciliation procedures and settlement of disputes."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "1",
    "description": "Short title, extent, and commencement of the Act."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "2",
    "description": "Definitions of consumer, goods, services, unfair trade practices, and defect."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "3",
    "description": "Right to safety against hazardous goods and services."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "4",
    "description": "Right to information about goods and services."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "5",
    "description": "Right to choose freely among goods and services."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "6",
    "description": "Right to be heard; complaints and grievances must be considered."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "7",
    "description": "Right to seek redressal for unfair trade practices."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "8",
    "description": "Right to consumer education and awareness of rights."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "9-12",
    "description": "Establishment, composition, and jurisdiction of consumer commissions."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "13-23",
    "description": "Complaint filing, hearings, evidence, interim orders, settlement, and execution."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "24-30",
    "description": "Penalties for false advertisements, offences by companies, enforcement powers, and investigations."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "31-33",
    "description": "Mediation provisions and settlement of disputes outside court."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "34-40",
    "description": "Powers of commissions to summon, call documents, enforce orders, and impose penalties."
  },
  {
    "act": "Consumer Protection Act, 2019",
    "section": "41-101",
    "description": "Miscellaneous provisions: administrative rules, consumer organizations, and transitional clauses."
  }


]

# Utility: save/read cases file
CASES_FILE = "cases.json"

def load_cases():
    try:
        with open(CASES_FILE, "r") as f:
            return json.load(f)
    except Exception:
        return []

def save_cases(cases):
    with open(CASES_FILE, "w") as f:
        json.dump(cases, f, indent=2)

# in-memory cases (persisted to cases.json on each add)
cases = load_cases()
print(f"Loaded {len(cases)} saved case(s).")


Loaded 0 saved case(s).


In [ ]:
# Cell 4 — Backend API with law lookup, case history, and AI prediction

from flask import Flask, request, jsonify
from flask_cors import CORS
import json, os, random, datetime, requests

app = Flask("judicio_backend")
CORS(app)

# Load lawData from Cell 3
lawData = [
    {
        "act": "Code of Civil Procedure, 1908",
        "section": "2-43",
        "description": "General provisions, definitions, and powers of civil courts."
    },
    {
        "act": "Code of Civil Procedure, 1908",
        "section": "9",
        "description": "Courts' jurisdiction to try all civil suits unless specifically barred."
    },
    {
        "act": "Code of Civil Procedure, 1908",
        "section": "11",
        "description": "Prevents a party from filing a case that has already been decided (res judicata)."
    },
    # ... add rest of dataset here from Cell 3 ...
]

# Synthetic dataset for case history
synthetic_cases = [
    {
        "id": 1,
        "title": "Ramesh vs State",
        "description": "Dispute regarding property ownership under CPC Section 9",
        "act": "Code of Civil Procedure, 1908",
        "section": "9",
        "verdict": "Jurisdiction upheld. Plaintiff allowed to proceed."
    },
    {
        "id": 2,
        "title": "Meera vs XYZ Ltd.",
        "description": "Consumer filed complaint for defective goods",
        "act": "Consumer Protection Act, 2019",
        "section": "3",
        "verdict": "In favor of consumer. Company fined ₹50,000."
    },
    {
        "id": 3,
        "title": "ABC Bank vs Ravi Traders",
        "description": "Financial creditor initiated insolvency proceedings",
        "act": "Insolvency and Bankruptcy Code, 2016",
        "section": "7",
        "verdict": "Corporate insolvency resolution process admitted."
    }
]

# Utility: load and save user-added cases
CASES_FILE = "cases.json"

def load_cases():
    try:
        with open(CASES_FILE, "r") as f:
            return json.load(f)
    except Exception:
        return []

def save_cases(cases):
    with open(CASES_FILE, "w") as f:
        json.dump(cases, f, indent=2)

# merge synthetic + saved
cases = synthetic_cases + load_cases()

# OpenAI API setup
OPENAI_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_CHAT_URL = "https://api.openai.com/v1/chat/completions"
MODEL_NAME = "gpt-4o-mini"  # or your fine-tuned model

HEADERS = {
    "Authorization": f"Bearer {OPENAI_KEY}",
    "Content-Type": "application/json"
}

# ----- API Routes -----

@app.route("/laws", methods=["GET"])
def get_laws():
    """Search law dataset by section or act"""
    query = request.args.get("q", "").lower()
    results = []
    for law in lawData:
        if query in law["section"].lower() or query in law["act"].lower():
            results.append(law)
    return jsonify(results)

@app.route("/cases", methods=["GET"])
def get_cases():
    """Return all case history"""
    return jsonify(cases)

@app.route("/cases", methods=["POST"])
def add_case():
    """Add new case and persist"""
    data = request.json
    new_case = {
        "id": len(cases) + 1,
        "title": data.get("title", f"Case {len(cases)+1}"),
        "description": data.get("description", ""),
        "act": data.get("act", ""),
        "section": data.get("section", ""),
        "verdict": data.get("verdict", "Pending")
    }
    cases.append(new_case)
    save_cases(cases[len(synthetic_cases):])  # save only user-added
    return jsonify({"message": "Case added", "case": new_case})

@app.route("/predict", methods=["POST"])
def predict_judgement():
    """Call OpenAI to generate predicted judgement"""
    data = request.json
    case_desc = data.get("description", "")

    if not OPENAI_KEY:
        return jsonify({"error": "OpenAI API key not set"}), 500

    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a legal AI assistant predicting judgements based on Indian laws."},
            {"role": "user", "content": f"Case Description: {case_desc}\nPredict likely judgement with reasoning."}
        ],
        "temperature": 0.4
    }

    try:
        resp = requests.post(OPENAI_CHAT_URL, headers=HEADERS, json=payload)
        result = resp.json()
        prediction = result["choices"][0]["message"]["content"]
        return jsonify({"prediction": prediction})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

print("✅ Judicio backend ready. Endpoints: /laws, /cases, /predict")


✅ Judicio backend ready. Endpoints: /laws, /cases, /predict


In [ ]:
# Cell 5 — Expose Flask app via ngrok
from pyngrok import ngrok
import nest_asyncio
from threading import Thread

nest_asyncio.apply()

# 🔑 Enter your ngrok token here
ngrok.set_auth_token("32umE5Wg3Xhilnfzh9bnl3UoXmF_3bijk82yDrm9vgaCv3DMv")  # replace with your actual token

# Set up a tunnel to the Flask server
public_url = ngrok.connect(5000)
print(f"🚀 Judicio backend is live at: {public_url}")

# Run Flask in background
def run_app():
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

thread = Thread(target=run_app)
thread.start()
https://5432ff5a9f19.ngrok-free.app/

🚀 Judicio backend is live at: NgrokTunnel: "https://5432ff5a9f19.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app 'judicio_backend'
 * Debug mode: off


In [ ]:
from IPython.core.display import display, HTML

html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>Judicio — Indian Legal Assistant</title>
  <style>
    body { font-family: Arial, sans-serif; padding: 20px; }
    main { background: white; padding: 20px; border-radius: 10px; max-width: 800px; margin:auto; box-shadow:0 4px 10px rgba(0,0,0,0.1); }
    h1 { text-align: center; }
    .tabs { display: flex; border-bottom: 2px solid #ddd; margin-bottom: 10px; }
    .tab { flex: 1; padding: 10px; cursor: pointer; text-align: center; }
    .tab.active { background: #f0f8ff; border-bottom: 2px solid #007bff; font-weight: bold; }
    .tab-content { display: none; }
    .tab-content.active { display: block; }
    label { font-weight: bold; margin-top: 10px; display: block; }
    input, textarea, select { width: 100%; padding: 8px; margin-top: 5px; border:1px solid #ccc; border-radius:5px; }
    button { margin-top: 10px; padding: 10px; width: 100%; background:#007bff; color:white; border:none; border-radius:5px; cursor:pointer; }
    button:hover { background:#0056b3; }
    #result, #caseResult, #predictResult { margin-top: 10px; padding: 10px; background:#f9f9f9; border-radius:5px; min-height:40px; }
  </style>
</head>
<body>
  <main>
    <h1> Judicio — Legal Assistant</h1>
    <div class="tabs">
      <div class="tab active" data-tab="law">Law Lookup</div>
      <div class="tab" data-tab="case">Case History</div>
      <div class="tab" data-tab="predict">Judgement Predictor</div>
    </div>

    <!-- Law Lookup -->
    <div id="law" class="tab-content active">
      <label>Choose Act:</label>
      <select id="actSelect">
        <option value="evidence">Indian Evidence Act, 1872</option>
        <option value="commercial">Commercial Courts Act, 2015</option>
        <option value="companies">Companies Act, 2013</option>
        <option value="CPC">Code of Civil Procedure(CPC),1908</option>
        <option value="consumer">Consumer Protection Act ,2019</option>
        <option value="arbitration">Arbitration and Conciliation Act,1996</option>

      </select>
      <label>Enter Section:</label>
      <input type="text" id="sectionInput" placeholder="e.g., 302" />
      <button id="searchBtn">Search Section</button>
      <div id="result"></div>
    </div>

    <!-- Case History -->
    <div id="case" class="tab-content">
      <label>Enter Case Number:</label>
      <input type="text" id="caseInput" placeholder="e.g., 123/2020" />
      <button id="caseBtn"> Get Case Details</button>
      <div id="caseResult"></div>
    </div>

    <!-- Judgement Predictor -->
    <div id="predict" class="tab-content">
      <label>Enter Case Description:</label>
      <textarea id="predictInput" rows="4" placeholder="Enter case facts..."></textarea>
      <button id="predictBtn">⚖ Predict Judgement</button>
      <div id="predictResult"></div>
    </div>
  </main>

  <script>
    //  Replace with your ngrok URL
    const NGROK_URL = "https://5432ff5a9f19.ngrok-free.app/";

    // Tabs
    document.querySelectorAll(".tab").forEach(tab=>{
      tab.addEventListener("click", ()=>{
        document.querySelectorAll(".tab").forEach(t=>t.classList.remove("active"));
        document.querySelectorAll(".tab-content").forEach(c=>c.classList.remove("active"));
        tab.classList.add("active");
        document.getElementById(tab.dataset.tab).classList.add("active");
      });
    });

    // Law Lookup
    document.getElementById('searchBtn').addEventListener('click', async ()=>{
      const act=document.getElementById('actSelect').value;
      const sec=document.getElementById('sectionInput').value.trim();
      const res=document.getElementById('result');
      res.innerHTML=" Loading...";
      try {
        const response=await fetch(`${NGROK_URL}/law_lookup`,{
          method:"POST",headers:{'Content-Type':'application/json'},
          body:JSON.stringify({ act: act, section: sec })
        });
        const data=await response.json();
        res.innerHTML=data.found? `<h3>Section ${sec}</h3><p>${data.description}</p>`: " Section not found";
      } catch(err){ res.innerHTML="Error connecting to backend"; console.error(err); }
    });

    // Case History
    document.getElementById('caseBtn').addEventListener('click', async ()=>{
      const caseNo=document.getElementById('caseInput').value.trim();
      const out=document.getElementById('caseResult');
      out.innerHTML=" Loading...";
      try {
        const response=await fetch(`${NGROK_URL}/case_details`,{
          method:"POST",headers:{'Content-Type':'application/json'},
          body:JSON.stringify({ case_no: caseNo })
        });
        const data=await response.json();
        out.innerHTML=data.found? `<h3>Case ${caseNo}</h3><p>${data.details}</p>`: " Case not found";
      } catch(err){ out.innerHTML="Error connecting to backend"; console.error(err); }
    });

    // Judgement Predictor
    document.getElementById('predictBtn').addEventListener('click', async ()=>{
      const desc=document.getElementById('predictInput').value.trim();
      const out=document.getElementById('predictResult');
      if(!desc){ out.textContent="Enter description"; return; }
      out.innerHTML=" Analyzing...";
      try {
        const response=await fetch(`${NGROK_URL}/predict_judgement`,{
          method:"POST",headers:{'Content-Type':'application/json'},
          body:JSON.stringify({ description: desc })
        });
        const data=await response.json();
        out.innerHTML=data.prediction || " No prediction available";
      } catch(err){ out.innerHTML=" Error connecting to backend"; console.error(err); }
    });
  </script>
</body>
</html>
"""

display(HTML(html_code))
